Create and Test Team Ratings for 2023 Season - Round by Round

In [1]:
import pandas as pd
import re
import numpy as np
from scipy import stats
from sklearn import metrics

import os
import warnings
warnings.filterwarnings('ignore')
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
pd.set_option('display.precision', 4)

In [2]:
player_stats = pd.read_csv("/Users/ciaran/Documents/Projects/AFL/data/scored_player_stats_v2.csv")
player_stats['Season'] = player_stats['Match_ID'].apply(lambda x: int(x[:4]))
player_stats['Player_Season'] = player_stats['Player'] + "_" + player_stats['Season'].astype(str)

player_stats['Score'] = player_stats['Behinds'] + player_stats['Goals']*6
player_stats['xScore_Diff'] = player_stats['Score'] - player_stats['xScore']

player_stats['Home_Team'] = player_stats['Match_ID'].apply(lambda x: x.split("_")[1]).str.replace(r"([A-Z])", r" \1").str.strip()
player_stats['Away_Team'] = player_stats['Match_ID'].apply(lambda x: x.split("_")[-1]).str.replace(r"([A-Z])", r" \1").str.strip()
player_stats['Opponent'] = np.where(player_stats['Team'] == player_stats['Home_Team'], player_stats['Away_Team'], player_stats['Home_Team'])

finals_round_map = {'F1':24,
                    'F2':25,
                    'F3':26,
                    'F4':27}
player_stats['Round_ID'] = player_stats['Round_ID'].astype(str)
player_stats['Round'] = (player_stats['Round_ID'].str[4:].replace(finals_round_map)).astype(int)
player_stats['Round_str'] = player_stats['Round_ID'].str[4:].replace(finals_round_map)
player_stats['Round_ID_num'] = (player_stats['Season'].astype(str) + player_stats['Round'].astype(str)).astype(int)

player_stats.tail()

,Match_ID,Team,Player,Round_ID,AFL_API_Player_ID,Player_Type,playerId,Age,Height,Weight,Number,Kicking_Foot,State_Of_Origin,Draft_Year,Debut_Year,Recruited_From,Draft_Position,Draft_Type,Photo_URL,Date_Of_Birth,Percent_Played,Behinds,Bounces,Centre_Bounces_Attended,Centre_Clearances,Clangers,Defensive_Contest_Losses,Defensive_Contest_Loss_Percentage,Defensive_One_On_One_Contests,Contested_Marks,Contested_Possession_Rate,Contested_Possessions,Offensive_One_On_One_Contests,Offensive_Contest_Wins,Offensive_Contest_Win_Percentage,Defensive_Half_Pressure_Acts,Disposal_Efficiency,Disposals,AFL_Fantasy_Points,Effective_Disposals,Effective_Kicks,Inside_50_Ground_Ball_Gets,Frees_Against,Frees_For,Goal_Accuracy,Goal_Assists,Goals,Ground_Ball_Gets,Handballs,Hit_Outs,Hit_Outs_To_Advantage,Hit_Outs_To_Advantage_Rate,Hit_Out_Win_Percentage,Inside_50s,Intercept_Marks,Intercepts,Kick_Efficiency,Kick_Ins,Kick_Ins_Played_On,Kicks,Kick_To_Handball_Ratio,Marks,Marks_Inside_50,Marks_On_Lead,Metres_Gained,One_Percenters,Pressure_Acts,Player_Rating_Points,Rebound_50s,Ruck_Contests,Score_Involvements,Score_Launches,Shots_At_Goal,Spoils,Stoppage_Clearances,Tackles,Tackles_Inside_50,Clearances,Possessions,Turnovers,Uncontested_Possessions,AFLCA_Player_ID,Coaches_Votes,Position,Team_Status,Position_Sub_Group,Position_Group,Year,Brownlow_Votes,Season,xScore,xT_created,xT_denied,vaep_value,offensive_value,defensive_value,exp_vaep_value,exp_offensive_value,exp_defensive_value,xT_received,xT_prevented,vaep_value_received,exp_vaep_value_received,Player_Season,Score,xScore_Diff,Home_Team,Away_Team,Opponent,Round,Round_str,Round_ID_num
28536,202324_WestCoast_Adelaide,West Coast,Ryan Maric,202324,Ryan_Maric,KEY_FORWARD,CD_I1029416,18,196,93,41,RIGHT,VIC,2023.0,2023.0,Drouin Vic)/Gippsland U18/Box Hill Hawks (VFL),1.0,other,https://s.afl.com.au/staticfile/AFL Tenant/AFL...,2004-09-06,71,0,0.0,NaN,0,0,NaN,NaN,NaN,0,NaN,2,NaN,NaN,NaN,NaN,77.8,9,49,NaN,NaN,NaN,0,2,0.0,1,0,NaN,3,0,NaN,NaN,NaN,0,NaN,1,NaN,NaN,NaN,6,NaN,5,0,NaN,114.0,0,NaN,NaN,1,NaN,3,NaN,0,NaN,0,2,0,0,10,2,8,NaN,NaN,Half Forward Flank Left,FINAL_TEAM,Half-Forward,Forward,NaN,NaN,2023,0.0000,-0.0587,0.0408,0.0525,-0.1360,0.1885,2.1304,0.9662,1.1643,0.2911,0.0000,0.4040,4.3952,Ryan Maric_2023,0,0.0000,West Coast,Adelaide,Adelaide,24,24,202324
28537,202324_WestCoast_Adelaide,West Coast,Sam Petrevski-Seton,202324,Sam_Petrevski-Seton,MEDIUM_FORWARD,CD_I994539,25,181,78,10,RIGHT,WA,2016.0,2017.0,Halls Creek (WA)/Claremont (WAFL)/Carlton,6.0,nationalDraft,https://s.afl.com.au/staticfile/AFL Tenant/AFL...,1998-02-19,61,0,0.0,NaN,0,3,NaN,NaN,NaN,0,NaN,6,NaN,NaN,NaN,NaN,83.3,18,64,NaN,NaN,NaN,0,1,0.0,1,0,NaN,12,0,NaN,NaN,NaN,0,NaN,4,NaN,NaN,NaN,6,NaN,3,0,NaN,87.0,1,NaN,NaN,2,NaN,2,NaN,0,NaN,0,3,0,0,18,4,12,NaN,NaN,Interchange,FINAL_TEAM,Interchange,Interchange,NaN,NaN,2023,0.0000,-0.2735,0.0312,-0.4014,-0.2771,-0.1243,-2.3624,-1.7040,-0.6584,-0.1613,0.1136,-0.0440,0.1591,Sam Petrevski-Seton_2023,0,0.0000,West Coast,Adelaide,Adelaide,24,24,202324
28538,202324_WestCoast_Adelaide,West Coast,Shannon Hurn,202324,Shannon_Hurn,MEDIUM_DEFENDER,CD_I240283,35,187,95,25,RIGHT,SA,2005.0,2006.0,Angaston (SA)/Central District (SANFL),13.0,nationalDraft,https://s.afl.com.au/staticfile/AFL Tenant/AFL...,1987-09-04,83,0,0.0,NaN,0,2,NaN,NaN,NaN,0,NaN,2,NaN,NaN,NaN,NaN,95.5,22,89,NaN,NaN,NaN,0,1,0.0,0,0,NaN,4,0,NaN,NaN,NaN,0,NaN,4,NaN,NaN,NaN,18,NaN,6,0,NaN,506.0,3,NaN,NaN,7,NaN,2,NaN,0,NaN,1,2,0,1,12,2,10,NaN,NaN,Back Pocket Right,FINAL_TEAM,Back-Pocket,Back,NaN,NaN,2023,0.0000,-0.3356,0.0329,0.6958,0.1442,0.5515,1.3235,2.1248,-0.8013,-1.5501,0.1773,-0.4591,-1.6297,Shannon Hurn_2023,0,0.0000,West Coast,Adelaide,Adelaide,24,24,202324
28539,202324_WestCoast_Adelaide,West Coast,Tim Kelly,202324,Tim_Kelly,MIDFIELDER,CD_I295898,29,184,83,11,RIGHT,WA,2017.0,2018.0,Palmyra (WA)/South Fremantle (WAFL)/Geelong,24.0,nationalDraft,https://s.afl.com.au/staticfile/AFL Tenant/AFL...,1994-07-26,84,0,2.0,NaN,4,5,NaN,NaN,NaN,0,NaN,15,NaN,NaN,Na

Pre-Season

In [3]:
def get_previous_round_id(round_id):
    
    finals_round_map = {'F1':24,
                        'F2':25,
                        'F3':26,
                        'F4':27}
    
    season = round_id[:4]
    if 'F' in round_id:
        round_num = finals_round_map[round_id[4:]]
    else:
        round_num = round_id[4:]
    
    previous_round_num = int(round_id[4:])-1
    previous_round_num = str(previous_round_num).zfill(2)
    if previous_round_num == 1:
        previous_season = int(season) - 1
    else:
        previous_season = season
    previous_round_id = str(previous_season) + str(previous_round_num)
    
    return previous_round_id

In [4]:
def get_score_standard_deviation(data):
    return data.groupby(['Match_ID', 'Team']).sum()['Score'].std()

In [5]:
def create_team_summary(player_stats, rounds = None):
    
    player_stats_copy = player_stats.copy()
    
    if rounds is not None:
        all_rounds = sorted(list(set(player_stats_copy['Round_ID'])))
        rating_rounds = all_rounds[-rounds:]
        player_stats_copy = player_stats_copy[player_stats_copy['Round_ID'].isin(rating_rounds)]
    
    team_for = player_stats_copy.groupby(['Team']).sum()[['Score', 'xScore','vaep_value', 'vaep_value_received', 'offensive_value', 'defensive_value', 'exp_vaep_value', 'exp_vaep_value_received', 'exp_offensive_value', 'exp_defensive_value']]
    team_for.columns = [x+"_F" for x in list(team_for)]
    team_against = player_stats_copy.groupby(['Opponent']).sum()[['Score', 'xScore', 'vaep_value', 'vaep_value_received', 'offensive_value', 'defensive_value', 'exp_vaep_value', 'exp_vaep_value_received', 'exp_offensive_value', 'exp_defensive_value']]
    team_against.columns = [x+"_A" for x in list(team_against)]
    team_games = player_stats_copy.groupby(['Match_ID', 'Team']).size().reset_index().groupby(['Team']).count()[['Match_ID']]
    team_games.columns = ['Games']
    team_summary = pd.concat([team_games, team_for, team_against], axis=1)
    
    for col in [x.replace("_F", "") for x in list(team_for)]:
        team_summary[col+"_pergame_F"] = team_summary[col+"_F"] / team_summary["Games"]
        team_summary[col+"_pergame_A"] = team_summary[col+"_A"] / team_summary["Games"]

        team_summary[col+"_diff"] = team_summary[col+"_F"] - team_summary[col+"_A"]
        team_summary[col+"_pergame_diff"] = team_summary[col+"_pergame_F"] - team_summary[col+"_pergame_A"]

    return team_summary

In [6]:
def get_teams(match_id):
    
    home_team = re.sub(r"\B([A-Z])", r" \1", match_id.split("_")[1])
    away_team = re.sub(r"\B([A-Z])", r" \1", match_id.split("_")[-1])

    return home_team, away_team

In [7]:
def calculate_match_projections(home_attack_rating, home_defence_rating, away_attack_rating, away_defence_rating):
    
    home_projected_score = (home_attack_rating + away_defence_rating)*0.5
    away_projected_score = (home_defence_rating + away_attack_rating)*0.5
    projected_margin = home_projected_score - away_projected_score
    
    return home_projected_score, away_projected_score, projected_margin

In [8]:
def get_match_ratings(team_summary, home_team, away_team):
    
    match_ratings_dict = {'home_attack_rating':team_summary.loc[home_team]['exp_vaep_value_pergame_F'],
                          'home_defence_rating':team_summary.loc[home_team]['exp_vaep_value_pergame_A'],
                          'away_attack_rating':team_summary.loc[away_team]['exp_vaep_value_pergame_F'],
                          'away_defence_rating':team_summary.loc[away_team]['exp_vaep_value_pergame_A']}
    
    return match_ratings_dict

In [9]:
def calculate_projected_probabilities(home_projected_score, away_projected_score, score_std=25):
    
    projected_margin = home_projected_score - away_projected_score
    projected_margin_distribution = stats.norm(loc = projected_margin, scale = score_std)
    
    away_team_win_probability = projected_margin_distribution.cdf(0)
    draw_probability = projected_margin_distribution.pdf(0)
    home_team_win_probability = 1 - away_team_win_probability - draw_probability
    
    return home_team_win_probability, draw_probability, away_team_win_probability
    

In [10]:
def get_match_prediction_metrics(projected_margin, margin):
    
    mae = abs(projected_margin - margin)
    mse = ((projected_margin)**2 + (margin)**2)
    rmse = ((projected_margin)**2 + (margin)**2)**0.5
    
    return mae, mse, rmse

In [11]:
def get_match_result(data, match_id):
    
    home_team, away_team = get_teams(match_id)
    match_player_stats = data[data['Match_ID'] == match_id]
    match_score = match_player_stats.groupby('Team').sum()[['Score', 'xScore']]
    home_score, home_xscore = match_score.loc[home_team]['Score'], match_score.loc[home_team]['xScore']
    away_score, away_xscore = match_score.loc[away_team]['Score'], match_score.loc[away_team]['xScore']
    margin, xmargin = home_score - away_score, home_xscore - away_xscore
    
    return home_score, away_score, margin

In [12]:
def calculate_match_projection_error(player_stats, match_id, match_ratings_dict, score_std = 25):
    
    home_projected_score, away_projected_score, projected_margin = calculate_match_projections(match_ratings_dict['home_attack_rating'], 
                                                                             match_ratings_dict['home_defence_rating'], 
                                                                             match_ratings_dict['away_attack_rating'], 
                                                                             match_ratings_dict['away_defence_rating'])
    
    home_team_win_probability, draw_probability, away_team_win_probability = calculate_projected_probabilities(home_projected_score, away_projected_score, score_std)
    home_score, away_score, margin = get_match_result(player_stats, match_id)
    
    mae, mse, rmse = get_match_prediction_metrics(projected_margin, margin)
    
    return mae, mse, rmse

In [13]:
def get_latest_rounds(round_id, player_stats, rounds):
    
    all_rounds = sorted(list(set(player_stats['Round_ID'])))
    rating_rounds = all_rounds[all_rounds.index(round_id)-rounds:all_rounds.index(round_id)]
    player_stats_latest = player_stats[player_stats['Round_ID'].isin(rating_rounds)]
    
    return player_stats_latest

In [14]:
def update_round_projections(round_id, player_stats, rounds, round_projection_dict, round_mae):
    
    previous_round_id = get_previous_round_id(round_id)
    player_stats_latest = get_latest_rounds(round_id, player_stats, rounds)
    team_summary = create_team_summary(player_stats_latest, rounds = rounds)
    team_summary = team_summary[['exp_vaep_value_pergame_F', 'exp_vaep_value_pergame_A', 'exp_vaep_value_pergame_diff']]

    round_projection_dict[round_id] = create_round_match_projections(player_stats, team_summary, round_id)
    round_mae[round_id] = np.mean(np.array(list({v['MAE'] for (k, v) in round_projection_dict[round_id].items()})))
    print("Round MAE: {}".format(round_mae[round_id]))
    print("Overall MAE: {}".format(np.mean(np.array(list(round_mae.values())))))
    return team_summary.sort_values('exp_vaep_value_pergame_diff', ascending = False)

In [15]:
def create_round_match_projections(player_stats, team_summary, round_id):
    
    player_stats_round = player_stats[player_stats['Round_ID'] == round_id]
    match_id_list = list(player_stats_round['Match_ID'].unique())

    match_projection_dict = {}
    for match_id in match_id_list:
        home_team, away_team = get_teams(match_id)
        match_ratings_dict = get_match_ratings(team_summary, home_team, away_team)
        home_projected_score, away_projected_score, projected_margin = calculate_match_projections(match_ratings_dict['home_attack_rating'], 
                                                                                                    match_ratings_dict['home_defence_rating'], 
                                                                                                    match_ratings_dict['away_attack_rating'], 
                                                                                                    match_ratings_dict['away_defence_rating'])
        home_team_win_probability, draw_probability, away_team_win_probability = calculate_projected_probabilities(home_projected_score, away_projected_score, score_std)
        home_score, away_score, margin = get_match_result(player_stats_round, match_id)
        mae, mse, rmse = calculate_match_projection_error(player_stats_round, match_id, match_ratings_dict, score_std = 25)

        match_projection_dict[match_id] = {}
        if home_projected_score > away_projected_score:
            match_projection_dict[match_id]['tip'] = home_team
            match_projection_dict[match_id]['projected_prob'] = home_team_win_probability
        else:
            match_projection_dict[match_id]['tip'] = away_team
            match_projection_dict[match_id]['projected_prob'] = away_team_win_probability
        match_projection_dict[match_id]['projected_margin'] = abs(projected_margin)
        if home_score > away_score:
            match_projection_dict[match_id]['result'] = home_team
        else:
            match_projection_dict[match_id]['result'] = away_team
        match_projection_dict[match_id]['actual_margin'] = abs(margin)   
        match_projection_dict[match_id]['MAE'] = mae
    
    return match_projection_dict
    

In [16]:
score_std = get_score_standard_deviation(player_stats)

In [17]:
round_projection_dict = {}
round_mae = {}
rounds = 10

202301

In [18]:
round_id = '202301'
player_stats_latest = player_stats[player_stats['Season'] == 2022]
team_summary = create_team_summary(player_stats_latest, rounds = rounds)
team_summary = team_summary[['exp_vaep_value_pergame_F', 'exp_vaep_value_pergame_A', 'exp_vaep_value_pergame_diff']]

round_projection_dict[round_id] = create_round_match_projections(player_stats, team_summary, round_id)
round_mae[round_id] = np.mean(np.array(list({v['MAE'] for (k, v) in round_projection_dict[round_id].items()})))
print("Round MAE: {}".format(round_mae[round_id]))
print("Overall MAE: {}".format(np.mean(np.array(list(round_mae.values())))))
team_summary.sort_values('exp_vaep_value_pergame_diff', ascending = False)

Round MAE: 30.9274858993962
Overall MAE: 30.9274858993962


,exp_vaep_value_pergame_F,exp_vaep_value_pergame_A,exp_vaep_value_pergame_diff
Geelong,97.5972,60.1562,37.4410
Richmond,103.6260,83.8944,19.7316
Sydney,96.0714,78.9025,17.1689
Melbourne,92.8762,81.9898,10.8864
Port Adelaide,88.8701,78.2676,10.6025
Western Bulldogs,83.6125,78.8754,4.7371
Brisbane Lions,90.7450,88.4753,2.2697
Fremantle,82.7621,84.6183,-1.8562
Carlton,85.5714,88.5205,-2.9491
St Kilda,73.1233,78.8976,-5.7744


202302

In [19]:
round_id = '202302'
update_round_projections(round_id, player_stats, rounds, round_projection_dict, round_mae)

Round MAE: 35.576048534553564
Overall MAE: 33.25176721697488


,exp_vaep_value_pergame_F,exp_vaep_value_pergame_A,exp_vaep_value_pergame_diff
Geelong,98.6166,66.9557,31.6608
Port Adelaide,100.8720,76.9784,23.8936
Sydney,98.0524,77.7449,20.3075
Richmond,102.9924,84.5656,18.4267
Melbourne,95.2197,79.5552,15.6645
Fremantle,85.5871,85.5930,-0.0058
Carlton,91.9053,92.2559,-0.3506
Western Bulldogs,83.1429,83.7953,-0.6524
Collingwood,89.3065,94.8560,-5.5496
St Kilda,78.6978,84.4191,-5.7213


202303

In [20]:
round_id = '202303'
update_round_projections(round_id, player_stats, rounds, round_projection_dict, round_mae)

Round MAE: 26.986897106906575
Overall MAE: 31.16347718028545


,exp_vaep_value_pergame_F,exp_vaep_value_pergame_A,exp_vaep_value_pergame_diff
Geelong,97.5937,67.7773,29.8164
Sydney,98.1927,76.8651,21.3276
Richmond,105.6618,87.8921,17.7697
Port Adelaide,97.0635,80.0527,17.0108
Melbourne,92.5107,79.5346,12.9761
Fremantle,85.3002,83.2904,2.0098
Collingwood,92.4037,92.3898,0.0140
Carlton,89.5683,93.1946,-3.6264
Western Bulldogs,81.7246,85.6429,-3.9182
St Kilda,82.9982,87.0801,-4.0819


202304

In [21]:
round_id = '202304'
update_round_projections(round_id, player_stats, rounds, round_projection_dict, round_mae)

Round MAE: 31.871101116934614
Overall MAE: 31.34038316444774


,exp_vaep_value_pergame_F,exp_vaep_value_pergame_A,exp_vaep_value_pergame_diff
Geelong,96.3415,70.4081,25.9334
Richmond,101.2497,86.0793,15.1704
Port Adelaide,100.7599,86.0460,14.7139
Sydney,96.5317,84.1790,12.3527
Melbourne,92.9513,82.4493,10.5020
Fremantle,90.4024,82.3066,8.0959
Carlton,97.1355,94.3673,2.7682
Collingwood,93.3198,91.6018,1.7180
Western Bulldogs,84.3222,84.5933,-0.2711
Greater Western Sydney,94.8038,98.6888,-3.8850


202305

In [22]:
round_id = '202305'
update_round_projections(round_id, player_stats, rounds, round_projection_dict, round_mae)

Round MAE: 28.035373001976353
Overall MAE: 30.67938113195346


,exp_vaep_value_pergame_F,exp_vaep_value_pergame_A,exp_vaep_value_pergame_diff
Geelong,98.5179,71.1659,27.3520
Port Adelaide,102.8668,82.9092,19.9576
Melbourne,91.9962,78.3282,13.6680
Sydney,94.0996,83.8823,10.2173
Richmond,96.0495,88.5719,7.4776
Carlton,96.4235,89.6325,6.7910
Fremantle,89.1332,82.7033,6.4299
St Kilda,92.3485,86.4056,5.9429
Collingwood,92.7283,91.4318,1.2964
Western Bulldogs,84.7374,83.9349,0.8025


202306

In [23]:
round_id = '202306'
update_round_projections(round_id, player_stats, rounds, round_projection_dict, round_mae)

Round MAE: 29.388271495686038
Overall MAE: 30.464196192575557


,exp_vaep_value_pergame_F,exp_vaep_value_pergame_A,exp_vaep_value_pergame_diff
Geelong,97.3908,72.3332,25.0577
Port Adelaide,97.4139,83.0702,14.3437
Melbourne,92.5335,82.2901,10.2434
Sydney,93.3332,84.4868,8.8464
St Kilda,96.3348,88.1328,8.2020
Essendon,105.2212,97.1033,8.1179
Collingwood,94.8932,89.6400,5.2532
Fremantle,91.2351,87.8814,3.3537
Adelaide,95.6365,92.7448,2.8917
Richmond,93.1216,93.1049,0.0167


202307

In [24]:
round_id = '202307'
update_round_projections(round_id, player_stats, rounds, round_projection_dict, round_mae)

Round MAE: 32.76034249388358
Overall MAE: 30.792217092762417


,exp_vaep_value_pergame_F,exp_vaep_value_pergame_A,exp_vaep_value_pergame_diff
Geelong,96.7085,73.7005,23.0079
Essendon,106.9643,93.4170,13.5474
Adelaide,100.5992,88.5823,12.0169
St Kilda,100.2739,88.3386,11.9353
Port Adelaide,98.9440,88.1865,10.7575
Collingwood,96.7861,88.2694,8.5166
Melbourne,91.6634,84.6493,7.0141
Brisbane Lions,96.0742,92.9646,3.1096
Sydney,89.3943,89.3792,0.0150
Western Bulldogs,84.2191,86.5943,-2.3752


202308

In [25]:
round_id = '202308'
update_round_projections(round_id, player_stats, rounds, round_projection_dict, round_mae)

Round MAE: 20.85408245604121
Overall MAE: 29.549950263172267


,exp_vaep_value_pergame_F,exp_vaep_value_pergame_A,exp_vaep_value_pergame_diff
Geelong,98.1368,73.3047,24.8321
Melbourne,94.5257,80.7757,13.7501
Adelaide,97.3045,84.7444,12.5601
St Kilda,95.6360,85.4810,10.1550
Port Adelaide,94.5713,85.2754,9.2959
Carlton,98.0067,89.2300,8.7767
Collingwood,94.6721,86.6199,8.0522
Essendon,102.7094,95.1055,7.6038
Brisbane Lions,96.0122,89.5663,6.4459
Sydney,89.7118,88.4934,1.2183


202309

In [26]:
round_id = '202309'
update_round_projections(round_id, player_stats, rounds, round_projection_dict, round_mae)

Round MAE: 39.905910065500876
Overall MAE: 30.700612463431


,exp_vaep_value_pergame_F,exp_vaep_value_pergame_A,exp_vaep_value_pergame_diff
Geelong,98.0431,73.5967,24.4464
Port Adelaide,96.8783,83.5242,13.3541
Melbourne,97.8156,86.2576,11.5581
St Kilda,92.8360,82.1686,10.6674
Brisbane Lions,98.8216,88.7110,10.1106
Adelaide,94.6695,86.3013,8.3683
Collingwood,92.9078,86.4783,6.4295
Carlton,95.1290,89.4809,5.6481
Essendon,98.7789,97.3457,1.4332
Sydney,88.2359,87.9027,0.3332


202310

In [27]:
round_id = '202310'
update_round_projections(round_id, player_stats, rounds, round_projection_dict, round_mae)

Round MAE: 30.94024523678825
Overall MAE: 30.724575740766728


,exp_vaep_value_pergame_F,exp_vaep_value_pergame_A,exp_vaep_value_pergame_diff
Geelong,95.0707,73.4393,21.6314
Brisbane Lions,102.0190,84.1928,17.8262
Port Adelaide,98.6814,81.5660,17.1154
Melbourne,96.8259,81.9690,14.8569
Collingwood,91.8563,81.0726,10.7838
Adelaide,93.3046,82.7393,10.5653
St Kilda,88.5480,82.1927,6.3553
Gold Coast,94.5085,88.9878,5.5207
Carlton,93.7176,89.1063,4.6114
Fremantle,90.5260,88.9974,1.5285


202311

In [28]:
round_id = '202311'
update_round_projections(round_id, player_stats, rounds, round_projection_dict, round_mae)

Round MAE: 19.264495571041323
Overall MAE: 29.68275027079169


,exp_vaep_value_pergame_F,exp_vaep_value_pergame_A,exp_vaep_value_pergame_diff
Brisbane Lions,102.4191,82.4252,19.9939
Port Adelaide,97.7565,80.1495,17.6070
Geelong,95.0592,80.5002,14.5590
Collingwood,92.6886,80.9382,11.7504
Melbourne,93.8834,82.7154,11.1680
St Kilda,90.3787,81.5305,8.8482
Western Bulldogs,84.4224,79.3196,5.1029
Adelaide,88.8614,83.9549,4.9065
Sydney,89.8153,86.7755,3.0398
Fremantle,92.4038,89.7225,2.6812


202312

In [29]:
round_id = '202312'
update_round_projections(round_id, player_stats, rounds, round_projection_dict, round_mae)

Round MAE: 21.28371698946209
Overall MAE: 28.982830830680893


,exp_vaep_value_pergame_F,exp_vaep_value_pergame_A,exp_vaep_value_pergame_diff
Brisbane Lions,103.4053,75.5943,27.8110
Geelong,93.5493,78.1675,15.3818
Port Adelaide,92.5992,80.0800,12.5192
Collingwood,90.6193,79.5175,11.1018
Western Bulldogs,89.5700,79.7110,9.8590
Melbourne,92.9536,83.9933,8.9603
St Kilda,89.2189,82.6058,6.6132
Adelaide,87.6154,81.9168,5.6986
Gold Coast,95.6238,91.0720,4.5519
Carlton,91.6310,87.8381,3.7928


202313

In [30]:
round_id = '202313'
update_round_projections(round_id, player_stats, rounds, round_projection_dict, round_mae)

Round MAE: 29.363209095760674
Overall MAE: 29.012090697225492


,exp_vaep_value_pergame_F,exp_vaep_value_pergame_A,exp_vaep_value_pergame_diff
Brisbane Lions,104.3092,75.3670,28.9422
Port Adelaide,100.5532,81.0009,19.5523
Geelong,94.6366,78.3585,16.2781
Melbourne,94.0305,79.7024,14.3282
Western Bulldogs,90.2803,76.8539,13.4263
Collingwood,92.7670,80.5018,12.2652
Gold Coast,96.5133,89.5935,6.9198
Adelaide,87.1948,82.7484,4.4465
St Kilda,86.6605,82.2834,4.3771
Fremantle,91.4728,91.3647,0.1081


202314

In [31]:
round_id = '202314'
update_round_projections(round_id, player_stats, rounds, round_projection_dict, round_mae)

Round MAE: 32.980000762508304
Overall MAE: 29.295512844745694


,exp_vaep_value_pergame_F,exp_vaep_value_pergame_A,exp_vaep_value_pergame_diff
Brisbane Lions,105.7597,77.2895,28.4702
Port Adelaide,100.9887,77.0585,23.9302
Geelong,96.8732,77.4153,19.4579
Melbourne,94.3077,78.8655,15.4422
Adelaide,90.8863,79.9007,10.9856
Western Bulldogs,89.7907,79.1383,10.6524
Collingwood,92.1188,84.5611,7.5576
Gold Coast,97.5873,91.2698,6.3175
St Kilda,81.8127,76.5823,5.2304
Fremantle,89.9293,91.5207,-1.5915


202315

In [32]:
round_id = '202315'
update_round_projections(round_id, player_stats, rounds, round_projection_dict, round_mae)

Round MAE: 44.89929573806301
Overall MAE: 30.335765037633514


,exp_vaep_value_pergame_F,exp_vaep_value_pergame_A,exp_vaep_value_pergame_diff
Brisbane Lions,106.3305,75.5574,30.7731
Port Adelaide,104.4109,75.6739,28.7370
Melbourne,93.1176,80.4664,12.6512
Western Bulldogs,90.9761,78.8633,12.1129
Collingwood,92.1205,81.6090,10.5115
Adelaide,91.6838,81.5859,10.0979
Geelong,92.3970,83.9040,8.4930
Gold Coast,98.0245,93.7971,4.2274
Carlton,88.3247,86.8629,1.4618
St Kilda,78.0313,77.3735,0.6578


202316

In [33]:
round_id = '202316'
update_round_projections(round_id, player_stats, rounds, round_projection_dict, round_mae)

Round MAE: 37.727383643780975
Overall MAE: 30.797741200517727


,exp_vaep_value_pergame_F,exp_vaep_value_pergame_A,exp_vaep_value_pergame_diff
Port Adelaide,106.3762,76.5035,29.8727
Brisbane Lions,98.2818,75.5196,22.7622
Melbourne,91.9424,76.3254,15.6170
Western Bulldogs,93.5060,77.9900,15.5160
Gold Coast,100.9179,88.6688,12.2491
Collingwood,89.5026,78.9579,10.5446
Carlton,89.4250,82.9601,6.4650
Adelaide,86.3684,81.6019,4.7665
Geelong,89.0688,84.3602,4.7086
Richmond,87.5407,85.6091,1.9316


202317

In [34]:
round_id = '202317'
update_round_projections(round_id, player_stats, rounds, round_projection_dict, round_mae)

Round MAE: 24.512817766605643
Overall MAE: 30.42803982205231


,exp_vaep_value_pergame_F,exp_vaep_value_pergame_A,exp_vaep_value_pergame_diff
Brisbane Lions,103.2378,76.3636,26.8741
Port Adelaide,102.8802,77.9923,24.8878
Melbourne,89.7572,69.9734,19.7839
Collingwood,93.2834,77.2311,16.0523
Carlton,91.6355,79.6536,11.9819
Adelaide,90.0838,78.7233,11.3605
Sydney,89.1533,78.9911,10.1622
Western Bulldogs,89.0498,79.0680,9.9818
St Kilda,76.5288,72.6781,3.8507
Fremantle,86.7017,85.1765,1.5252


202318

In [35]:
round_id = '202318'
update_round_projections(round_id, player_stats, rounds, round_projection_dict, round_mae)

Round MAE: 29.76040906561913
Overall MAE: 30.39094922447269


,exp_vaep_value_pergame_F,exp_vaep_value_pergame_A,exp_vaep_value_pergame_diff
Brisbane Lions,106.0574,74.5964,31.4610
Port Adelaide,108.4585,79.5724,28.8861
Collingwood,95.5367,77.6122,17.9245
Melbourne,84.4726,71.1015,13.3711
Western Bulldogs,88.9191,81.2046,7.7145
Adelaide,91.4280,84.7646,6.6634
Carlton,87.8971,82.0686,5.8285
St Kilda,77.4171,72.5640,4.8531
Sydney,85.2687,80.6277,4.6410
Fremantle,86.1358,83.8248,2.3110


202319

In [36]:
round_id = '202319'
update_round_projections(round_id, player_stats, rounds, round_projection_dict, round_mae)

Round MAE: 16.78125344605202
Overall MAE: 29.67464944666107


,exp_vaep_value_pergame_F,exp_vaep_value_pergame_A,exp_vaep_value_pergame_diff
Brisbane Lions,106.6688,75.8044,30.8645
Port Adelaide,105.6259,83.7781,21.8478
Collingwood,97.3185,77.9595,19.3589
Melbourne,81.9009,69.6496,12.2514
Carlton,91.6896,81.6571,10.0325
Adelaide,91.7535,83.7177,8.0358
Western Bulldogs,89.9208,82.9202,7.0006
Sydney,87.5021,81.2575,6.2446
Geelong,86.1543,82.7229,3.4314
Greater Western Sydney,81.7632,79.5720,2.1912


202320

In [37]:
round_id = '202320'
update_round_projections(round_id, player_stats, rounds, round_projection_dict, round_mae)

Round MAE: 25.148231009831424
Overall MAE: 29.44832852481959


,exp_vaep_value_pergame_F,exp_vaep_value_pergame_A,exp_vaep_value_pergame_diff
Brisbane Lions,102.1761,74.9592,27.2169
Port Adelaide,104.1877,84.5200,19.6677
Carlton,96.4999,78.9629,17.5371
Collingwood,95.8108,82.4580,13.3529
Sydney,90.5190,79.4049,11.1141
Greater Western Sydney,90.4545,79.4590,10.9956
Western Bulldogs,91.1806,81.9848,9.1958
Melbourne,83.4586,75.0978,8.3608
St Kilda,81.3767,75.9941,5.3826
Adelaide,93.3435,89.1270,4.2166


202321

In [38]:
round_id = '202321'
update_round_projections(round_id, player_stats, rounds, round_projection_dict, round_mae)

Round MAE: 22.343135812214097
Overall MAE: 29.10998601469552


,exp_vaep_value_pergame_F,exp_vaep_value_pergame_A,exp_vaep_value_pergame_diff
Brisbane Lions,101.1718,80.7040,20.4678
Carlton,97.4137,78.4687,18.9450
Greater Western Sydney,91.0372,76.0057,15.0315
Melbourne,88.6229,73.8579,14.7650
Port Adelaide,105.2364,91.5167,13.7196
Adelaide,102.3990,89.5687,12.8303
Collingwood,95.3167,83.3717,11.9450
Sydney,93.1714,82.4506,10.7208
St Kilda,79.9061,75.3581,4.5480
Western Bulldogs,89.0562,85.5339,3.5223


202322

In [39]:
round_id = '202322'
update_round_projections(round_id, player_stats, rounds, round_projection_dict, round_mae)

Round MAE: 23.55839191173264
Overall MAE: 28.857640828197205


,exp_vaep_value_pergame_F,exp_vaep_value_pergame_A,exp_vaep_value_pergame_diff
Brisbane Lions,101.6860,80.6337,21.0523
Carlton,96.3980,75.8998,20.4982
Adelaide,104.2276,86.7073,17.5203
Greater Western Sydney,90.4817,75.6750,14.8068
Melbourne,87.4094,72.7507,14.6587
Port Adelaide,106.2557,96.4091,9.8466
Sydney,92.3315,82.6444,9.6871
Western Bulldogs,90.9759,82.6241,8.3519
Collingwood,93.2040,86.5850,6.6191
Geelong,87.0677,81.8012,5.2665


202323

In [40]:
round_id = '202323'
update_round_projections(round_id, player_stats, rounds, round_projection_dict, round_mae)

Round MAE: 29.90137976829087
Overall MAE: 28.90302078211432


,exp_vaep_value_pergame_F,exp_vaep_value_pergame_A,exp_vaep_value_pergame_diff
Carlton,100.1338,76.0314,24.1025
Adelaide,104.5013,84.3277,20.1736
Brisbane Lions,100.5817,82.6688,17.9129
Greater Western Sydney,90.0976,76.9175,13.1801
Melbourne,87.5410,76.4865,11.0545
Sydney,91.9967,81.3789,10.6178
Port Adelaide,102.6765,92.9898,9.6867
Western Bulldogs,89.0814,82.8727,6.2087
St Kilda,78.2519,72.2465,6.0054
Geelong,88.1127,83.7873,4.3253


202324

In [41]:
round_id = '202324'
update_round_projections(round_id, player_stats, rounds, round_projection_dict, round_mae)

Round MAE: 28.265758009291986
Overall MAE: 28.876468166580057


,exp_vaep_value_pergame_F,exp_vaep_value_pergame_A,exp_vaep_value_pergame_diff
Carlton,99.8665,75.4271,24.4394
Brisbane Lions,101.3622,82.2778,19.0845
Greater Western Sydney,95.9020,78.1049,17.7971
Sydney,96.5375,83.2222,13.3153
Melbourne,85.4065,73.6813,11.7253
Adelaide,96.8168,86.5251,10.2917
Western Bulldogs,89.5850,80.5466,9.0384
St Kilda,82.2219,73.5852,8.6367
Port Adelaide,100.2039,91.8267,8.3772
Collingwood,93.3839,88.1215,5.2623
